In [11]:
df.head()

,Rank,Time,Name,Country,Date of Birth,Place,City,Date,Gender,Event
0,1,2:00:35,Kelvin Kiptum,KEN,02.12.99,1,Chicago,08.10.2023,Men,Marathon
1,2,2:01:09,Eliud Kipchoge,KEN,05.11.84,1,Berlin,25.09.2022,Men,Marathon
2,3,2:01:25,Kelvin Kiptum,KEN,02.12.99,1,London,23.04.2023,Men,Marathon
3,4,2:01:39,Eliud Kipchoge,KEN,05.11.84,1,Berlin,16.09.2018,Men,Marathon
4,5,2:01:41,Kenenisa Bekele,ETH,13.06.82,1,Berlin,29.09.2019,Men,Marathon


In [40]:
df['Datetime'] = df['Time'].apply(pd.to_datetime, errors='coerce')
df['City'] = df['City'].replace('New York City', 'New York')

df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
df['Year'] = df['Date'].dt.year

In [41]:
marathons_df = df.loc[
    (df['Event'] == 'Marathon')
]

In [42]:
unique_cities_sorted = marathons_df['City'].value_counts().sort_values(ascending=False)
major_cities = unique_cities_sorted[unique_cities_sorted > 50]
major_cities

London         649
Berlin         452
Tokyo          429
Chicago        368
Boston         359
Paris          346
Valencia       344
Amsterdam      321
Rotterdam      293
New York       292
Osaka          290
Seoul          284
Dubai          282
Frankfurt      269
Nagoya         238
Hamburg        193
Sevilla        176
Fukuoka        174
Otsu           169
Praha          143
Beijing        139
Roma           129
Eindhoven      122
Toronto        121
Wien           121
Milano         121
Barcelona      106
Xiamen         101
Shanghai        92
Houston         88
Daegu           88
Ljubljana       70
Los Angeles     70
Enschede        62
Lisboa          62
Torino          61
Kosice          56
Ottawa          52
Name: City, dtype: int64

In [43]:
marathons_df = marathons_df[marathons_df['City'].isin(major_cities.index)]

In [44]:
idx = marathons_df.groupby(['City', 'Gender'])['Datetime'].idxmin()
best_times = marathons_df.loc[idx][['Time', 'Name', 'Country', 'City', 'Gender', 'Date', 'Year']]
best_times

,Time,Name,Country,City,Gender,Date,Year
37,2:03:39,Tamirat Tola,ETH,Amsterdam,Men,2021-10-17,2021
18211,2:17:20,Almaz Ayana,ETH,Amsterdam,Women,2022-10-16,2022
200,2:05:06,Marius Kimutai,BRN,Barcelona,Men,2023-03-19,2023
18310,2:19:44,Zeineba Yimer,ETH,Barcelona,Women,2023-03-19,2023
625,2:06:32,Samuel Wanjiru,KEN,Beijing,Men,2008-08-24,2008
...,...,...,...,...,...,...,...
18195,2:14:58,Amane Beriso,ETH,Valencia,Women,2022-04-12,2022
204,2:05:08,Samuel Nyamae,KEN,Wien,Men,2023-04-23,2023
18423,2:20:59,Vibian Chepkurui,KEN,Wien,Women,2022-04-24,2022
539,2:06:19,Moses Mosop,KEN,Xiamen,Men,2015-03-01,2015


In [45]:
best_times['Date'] = best_times['Date'].dt.strftime('%Y-%m-%d')
best_times.to_json(
    '../public/best_times.json', 
    orient='records',
    indent=2
)

In [46]:
idx = marathons_df.groupby(['City', 'Gender'])['Date'].idxmax()
latest_times = marathons_df.loc[idx][['Time', 'Name', 'Country', 'City', 'Gender', 'Date', 'Year']]
latest_times

,Time,Name,Country,City,Gender,Date,Year
78,2:04:18,Joshua Belet,KEN,Amsterdam,Men,2023-10-15,2023
18242,2:18:21,Meseret Belete,ETH,Amsterdam,Women,2023-10-15,2023
200,2:05:06,Marius Kimutai,BRN,Barcelona,Men,2023-03-19,2023
18310,2:19:44,Zeineba Yimer,ETH,Barcelona,Women,2023-03-19,2023
1320,2:07:42,Yihunilign Adane,ETH,Beijing,Men,2023-10-29,2023
...,...,...,...,...,...,...,...
18199,2:15:51,Worknesh Degefa,ETH,Valencia,Women,2023-03-12,2023
204,2:05:08,Samuel Nyamae,KEN,Wien,Men,2023-04-23,2023
19089,2:24:12,Magdalene Masai,KEN,Wien,Women,2023-04-23,2023
1631,2:08:04,Philimon Kipchumba,KEN,Xiamen,Men,2023-02-04,2023


In [47]:
latest_times['Date'] = latest_times['Date'].dt.strftime('%Y-%m-%d')
latest_times.to_json(
    '../public/latest_times.json', 
    orient='records',
    indent=2,
)